In [1]:
import pandas as pd
from pandas.tseries.offsets import BDay 
import requests
from bs4 import BeautifulSoup
from datetime import time, timedelta, date

#Alpha_Vantage is a stock data provider.  This library makes api calls much easier 
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries  

## First, we'll read in the data output by the scraper and do any cleaning that didn't survive the "to_csv" part of the last notebook.  

In [2]:
df = pd.read_csv('AllHoldings.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390168 entries, 0 to 390167
Data columns (total 12 columns):
manager            390168 non-null object
filing_date        390168 non-null object
report_date        390168 non-null object
sh_name            390168 non-null object
sh_class           390168 non-null object
cusip              390168 non-null object
val1000            390168 non-null int64
share_count        390168 non-null int64
share_or_prin      390168 non-null object
discretion         390168 non-null object
sole_vote_amt      390168 non-null int64
shared_vote_amt    390168 non-null int64
dtypes: int64(4), object(8)
memory usage: 35.7+ MB


In [4]:
df['report_date'] = pd.to_datetime(df.report_date)

In [5]:
df['filing_date'] = pd.to_datetime(df.filing_date)

## The next step is to return only the top 20 holdings, aggregated by date and Cusip/Share.   

In [6]:
# Sums up the holdings by date/cusip and returns a flattened dataframe with report_date as the index
summed_df = df.groupby(['report_date', 'cusip', 'sh_name'])['report_date', 'cusip', 'sh_name', 'val1000'].sum().reset_index()
summed_df.set_index('report_date')

,cusip,sh_name,val1000
report_date,,,
2013-06-30,000360206,AAON INC,1179
2013-06-30,000361105,AAR CORP,2039
2013-06-30,000375204,ABB LTD,4802
2013-06-30,00081T108,ACCO BRANDS CORP,65
2013-06-30,000957100,ABM INDS INC,1731
2013-06-30,00101J106,THE ADT CORPORATION,12867
2013-06-30,001031103,AEP INDS INC,536
2013-06-30,00104Q107,AFC ENTERPRISES INC,4421
2013-06-30,001055102,AFLAC INC,75176


In [7]:
#Sets variables that will be used to loop through each report_date and select the top 20 by notional value
looper = summed_df.report_date.value_counts().index
top_df = pd.DataFrame(columns=['report_date', 'cusip', 'sh_name', 'val1000'])

In [8]:
for x in looper:
    top_df = top_df.append(summed_df[(summed_df.report_date == x)].nlargest(20, 'val1000'))

In [9]:
top_df = top_df.sort_values(['report_date', 'val1000'], ascending=[False, False]).reset_index().drop('index', axis=1)

## Now, the goal is to pull back stock tickers for all of the stocks that appear in our 420 row dataframe.  

#### This is a somewhat complicated task as stocks can undergo certain changes (which is why CUSIPS are often used as an identifier.

#### Fidelity seems to be the only free source of a cusip lookup, so I will first scrape there.  If there aren't too many NaNs after scraping Fidelity, I'll just figure out the rest of the tickers by hand.  If there are a lot, then I'll try some other automated solution.

In [ ]:
# creates a list of unique cusips.
cusips = list(top_df.cusip.unique())

In [ ]:
cusips[:5]

In [ ]:
url1 = 'https://quotes.fidelity.com/mmnet/SymLookup.phtml?reqforlookup=REQUESTFORLOOKUP&productid=mmnet&isLoggedIn=mmnet&rows=50&for=stock&by=cusip&criteria='
url2 = '&submit=Search'
ticker = []

for cusip in cusips:
    response = requests.get(url1+cusip+url2, verify=False)
    time.sleep(2)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        ticker.append(soup.find('td', {'align':'center'}).get_text())
    except:
        ticker.append(cusip+'Not Found')

In [10]:
#cusip_ticker = pd.DataFrame({'cusip':cusips, 'tickers':ticker})
cusip_ticker = pd.read_csv('cusip_ticker.csv')  
cusip_ticker.columns=['d', 'cusip', 'ticker']
cusip_ticker.drop('d', axis=1)

,cusip,ticker
0,949746101,WFC
1,500754106,KHC
2,191216100,KO
3,459200101,IBM
4,025816109,AXP
5,037833100,AAPL
6,451100101,IEP
7,78462F953,SPY
8,594918104,MSFT
9,742718109,PG


In [11]:
cusip_ticker[cusip_ticker.ticker.str.contains('Not')]
cusip_ticker.iloc[7,2] = 'SPY' 
cusip_ticker.iloc[10,2] = 'SPY' 
cusip_ticker.iloc[14,2] = 'VRX' #Company acquired by Bausch and Lomb 
cusip_ticker.iloc[20,2] = 'AAPL' #Looks like a miskey by the 13F filer
cusip_ticker.iloc[21,2] = 'DTV' #Company acquired by AT&T
cusip_ticker.iloc[29,2] = 'AGN' #Company acquired
cusip_ticker.iloc[33,2] = 'EMC' #Company acquired
cusip_ticker.iloc[46,2] = 'QQQ' #Fidelity tool not able to handle ETFs
cusip_ticker.iloc[48,2] = 'PCLN' #Company acquired by Bausch and Lomb 
cusip_ticker.iloc[56,2] = 'FRX' #Company acquired 
cusip_ticker.iloc[57,2] = 'MDT' #Company acquired 
cusip_ticker.iloc[58,2] = 'DWDP' #Company acquired 
cusip_ticker.iloc[60,2] = 'DELL' #Company went private 
cusip_ticker.iloc[61,2] = 'YHOO' #Company acquired 
cusip_ticker.iloc[64,2] = 'FDML' #Company went private 


In [12]:
cusip_ticker.head()

,d,cusip,ticker
0,0,949746101,WFC
1,1,500754106,KHC
2,2,191216100,KO
3,3,459200101,IBM
4,4,025816109,AXP


In [13]:
%env ALPHAVANTAGE_API_KEY = XXXXXXXXXXXX

env: ALPHAVANTAGE_API_KEY=XXXXXXXXXXXX


In [14]:
API = %env ALPHAVANTAGE_API_KEY
ts = TimeSeries(key='API', output_format='pandas', indexing_type='date')

In [15]:
alpha_data, alpha_meta_data = ts.get_daily_adjusted(symbol='WFC', outputsize='full')

SSLError: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=WFC&outputsize=full&apikey=API&datatype=json (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

## The above "alpha_data" dataframe came from the alphavantage API.  It had survivorship bias, so I instead just sourced the data directly from work, in the "prices" dataframe below.  This dataframe has unadjusted close prices only, but given the infrequency of my strategy's trading, the likelihood of corporate actions having a significant effect is low.

In [16]:
prices = pd.read_csv('13F_prices.txt', index_col=0, header=None, names=['date', 'ticker', 'price'])
prices.reset_index(inplace=True)

In [17]:
top_df = pd.merge(top_df, cusip_ticker, how='left', on='cusip').drop('d', axis=1)

In [18]:
top_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 5 columns):
report_date    420 non-null datetime64[ns]
cusip          420 non-null object
sh_name        420 non-null object
val1000        420 non-null object
ticker         420 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 19.7+ KB


In [19]:
top_df['buy_date'] = top_df['report_date'].apply(lambda x: x +  timedelta(days=44) + BDay(1))

In [20]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71214 entries, 0 to 71213
Data columns (total 3 columns):
date      71214 non-null object
ticker    71214 non-null object
price     71214 non-null float64
dtypes: float64(1), object(2)
memory usage: 1.6+ MB


In [22]:
prices['date'] = pd.to_datetime(prices.date)

In [23]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71214 entries, 0 to 71213
Data columns (total 3 columns):
date      71214 non-null datetime64[ns]
ticker    71214 non-null object
price     71214 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ MB


In [21]:
top_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 6 columns):
report_date    420 non-null datetime64[ns]
cusip          420 non-null object
sh_name        420 non-null object
val1000        420 non-null object
ticker         420 non-null object
buy_date       420 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(4)
memory usage: 23.0+ KB


In [24]:
top_df.head()

,report_date,cusip,sh_name,val1000,ticker,buy_date
0,2018-06-30,037833100,APPLE INC,54835353,AAPL,2018-08-14
1,2018-06-30,78462F103,SPDR S&P 500 ETF TR,29281034,SPY,2018-08-14
2,2018-06-30,949746101,WELLS FARGO & CO NEW,25059643,WFC,2018-08-14
3,2018-06-30,060505104,BANK AMER CORP,21612437,BAC,2018-08-14
4,2018-06-30,500754106,KRAFT HEINZ CO,20906754,KHC,2018-08-14


In [25]:
prices.head()

,date,ticker,price
0,2013-07-01,AAPL,409.220001
1,2013-07-01,AGN,125.440002
2,2013-07-01,AIG,45.090000
3,2013-07-01,AMGN,97.489998
4,2013-07-01,AMZN,282.100000


In [26]:
pd.merge(top_df, prices, how='left', left_on=['ticker', 'buy_date'], right_on=['ticker', 'date'])

,report_date,cusip,sh_name,val1000,ticker,buy_date,date,price
0,2018-06-30,037833100,APPLE INC,54835353,AAPL,2018-08-14,NaT,NaN
1,2018-06-30,78462F103,SPDR S&P 500 ETF TR,29281034,SPY,2018-08-14,NaT,NaN
2,2018-06-30,949746101,WELLS FARGO & CO NEW,25059643,WFC,2018-08-14,NaT,NaN
3,2018-06-30,060505104,BANK AMER CORP,21612437,BAC,2018-08-14,NaT,NaN
4,2018-06-30,500754106,KRAFT HEINZ CO,20906754,KHC,2018-08-14,NaT,NaN
5,2018-06-30,191216100,COCA COLA CO,18764843,KO,2018-08-14,NaT,NaN
6,2018-06-30,023135106,AMAZON COM INC,18040550,AMZN,2018-08-14,NaT,NaN
7,2018-06-30,025816109,AMERICAN EXPRESS CO,15349606,AXP,2018-08-14,NaT,NaN
8,2018-06-30,451100101,ICAHN ENTERPRISES LP,11826854,IEP,2018-08-14,NaT,NaN
9,2018-06-30,594918104,MICROSOFT CORP,10012998,MSFT,2018-08-14,NaT,NaN


### I found it weird that the above didn't work.  I spent a lot of time converting the datetime objects using various modules from pd.datetime and datetime.  Eventually, I figured out that there was whitespace in the prices.ticker column.

In [27]:
prices['ticker']=prices['ticker'].str.strip()

In [61]:
merged = pd.merge(top_df, prices, how='left', left_on=['ticker', 'buy_date'], right_on=['ticker', 'date'])

In [62]:
bad_tickers = merged[merged['price'].isna()].ticker.unique()
bad_dates = merged[merged.date.isna()].buy_date.unique()

In [63]:
bad_tickers

array(['IEP', 'QQQ', 'IWM', 'GOOG', 'PCLN', 'VWO', 'EMC', 'AGN', 'VRX',
       'DTV', 'SPY', 'DWDP'], dtype=object)

In [64]:
prices[(prices.date.isin(bad_dates))&(prices.ticker=='IEP')]

,date,ticker,price
26629,2015-05-15,IEP,91.85


In [72]:
merged_nulls = merged[merged['price'].isna()]
merged_nulls = merged_nulls.drop(['date', 'price'], axis=1)

### Based on the way I pulled price data, it's possible that just shifting by a day might get me prices close enough to what I need

In [73]:
merged_nulls.buy_date = merged_nulls.buy_date.apply(lambda x: x + BDay(1))

In [74]:
merged_nulls = pd.merge(merged_nulls, prices, how='left', left_on=['ticker', 'buy_date'], right_on=['ticker', 'date'])

In [75]:
merged_nulls.sort_values(by='ticker')

,report_date,cusip,sh_name,val1000,ticker,buy_date,date,price
27,2015-12-31,G0177J108,ALLERGAN PLC,4706704,AGN,2016-02-16,NaT,NaN
32,2015-06-30,25490A309,DIRECTV,4779892,DTV,2015-08-17,NaT,NaN
43,2014-09-30,25490A309,DIRECTV,3697653,DTV,2014-11-17,NaT,NaN
39,2014-12-31,25490A309,DIRECTV,3951832,DTV,2015-02-17,NaT,NaN
36,2015-03-31,25490A309,DIRECTV,3891948,DTV,2015-05-18,NaT,NaN
44,2014-09-30,260543103,DOW CHEM CO,3539742,DWDP,2014-11-17,NaT,NaN
25,2016-03-31,268648102,E M C CORP MASS,3939809,EMC,2016-05-17,NaT,NaN
23,2016-06-30,268648102,E M C CORP MASS,4264695,EMC,2016-08-16,NaT,NaN
8,2017-12-31,02079K107,ALPHABET INC,4300285,GOOG,2018-02-15,NaT,NaN
48,2014-03-31,451100101,ICAHN ENTERPRISES LP,10566566,IEP,2014-05-16,NaT,NaN


## There aren't too many of these, so I'm just going to pull them from Yahoo